In [1]:
import traceback

import pandas as pd
from IPython.display import HTML

try:
	from arcgis.gis import GIS
	from arcgis.network import ClosestFacilityLayer
	from arcgis import geocoding
	from arcgis.features import Feature, FeatureSet
except:
	import os
	print('arcgis not installed')
	command = 'conda install -c esri arcgis'
	os.system(command)
	# print()

# Connect to your GIS
gis = GIS("home")

ModuleNotFoundError: No module named 'arcgis'

In [ ]:
try:
    analysis_url = gis.properties.helperServices.closestFacility.url
    print("Closest Facility URL successfully retrieved")
except Exception as e:
    traceback.print_exc()
    raise Exception(f"Network Analysis is not properly configured "\
                    f"on your Organization: {e}")

In [ ]:
cf_layer = ClosestFacilityLayer(analysis_url, gis = gis)

In [ ]:
hospitals_addresses = [
    'San Gorgonio Memorial Hospital,600 N. Highland Springs Ave.,'\
        'Banning,CA,92220',
    'Desert Regional Medical Center,1150 N. Indian Canyon Dr.,'\
        'Palm Springs,CA,92262',
    'JFK Memorial Hospital,47111 Monroe St.,Indio,CA,92201',
    'Hi-Desert Medical Center,6601 White Feather Rd.,'\
        'Joshua Tree,CA,92252',
    'Eisenhower Health,39000 Bob Hope Dr.,Rancho Mirage,CA,92270']

In [ ]:
hosp_feat_list = []

for address in hospitals_addresses:
    hit = geocoding.geocode(address)[0]
    hosp_feat = Feature(geometry=hit['location'],
                        attributes=hit['attributes'])
    hosp_feat_list.append(hosp_feat)

In [ ]:
hospitals_fset = FeatureSet(features=hosp_feat_list, 
                            geometry_type='esriGeometryPoint', 
                            spatial_reference={'latestWkid': 4326})

In [ ]:
map1 = gis.map('Palm Springs, CA')
map1

In [ ]:
map1.zoom = 8


In [ ]:
map1.draw(hospitals_fset,
          symbol={"type": "esriSMS","style": "esriSMSSquare",
                  "color": [76,115,0,255],"size": 8,})

In [ ]:
# Create Incidents Layer
# Similarly, let us create the incident layer

incident_coords = '-116.391287,33.808053'
reverse_geocode = geocoding.reverse_geocode(
    {"x": incident_coords.split(',')[0], 
     "y": incident_coords.split(',')[1]})

incident_feature = Feature(geometry=reverse_geocode['location'], 
                           attributes=reverse_geocode['address'])
                           


In [ ]:
incident_fset = FeatureSet([incident_feature],
                           geometry_type='esriGeometryPoint',
                           spatial_reference={'latestWkid': 4326})
                           


In [ ]:
# Add the incident to the map

map1.draw(incident_fset,
          symbol={"type": "esriSMS","style": "esriSMSDiamond",
                  "size": 12, "color":[255,0,0,255]})
                  


In [ ]:
# Solve for Closest Facility
# By default the closest facility service would return only the closest location, so we need to specify explicitly the default_target_facility_count parameter as well as return_facilities to get multiple results.

result = cf_layer.solve_closest_facility(
    incidents=incident_fset,
    facilities=hospitals_fset,
    default_target_facility_count=5,
    return_facilities=True,
    impedance_attribute_name='TravelTime',
    accumulate_attribute_names=['Kilometers','TravelTime'])
    


In [ ]:
# Inspect the result dictionary

result.keys()


In [ ]:

# dict_keys(['messages', 'routes', 'facilities'])
# Let's use the routes dictionary to construct line features out of the routes to display on the map. From the routes dictionary will find out the proper key to get each route: the features key.

result['routes'].keys()


In [ ]:

# dict_keys(['fieldAliases', 'geometryType', 'spatialReference', 'features'])
result['routes']['features'][0].keys()


In [ ]:

# dict_keys(['attributes', 'geometry'])
# Construct line features out of the routes that are returned.

line_feat_list = []
for line_dict in result['routes']['features']:
    f1 = Feature(line_dict['geometry'], line_dict['attributes'])
    line_feat_list.append(f1)
routes_fset = FeatureSet(line_feat_list, 
    geometry_type=result['routes']['geometryType'],
    spatial_reference= result['routes']['spatialReference'])
# Add the routes back to the map. The route to the closest hospital is in red

map1.draw(routes_fset)
# Analyze Results in a Table
# Since we parsed the routes as a FeatureSet, we can display the attributes easily as a pandas DataFrame. We'll sort the dataframe according to the Name attribute of the solve_closest_facility task results.


In [ ]:

df1 = routes_fset.sdf.sort_values(by=['Name'])
df1


In [ ]:

# Add the hospital addresses and incident address to this table and display only the relevant columns

df1['facility_address'] = hospitals_addresses
df1['incident_address'] = [incident_feature.attributes['Match_addr'] \
                          for i in range(len(hospitals_addresses))]
pd.set_option('max_colwidth', 80)


In [ ]:

df1[['facility_address','incident_address','Total_Miles',
     'Total_TravelTime']].sort_values(by=['Total_TravelTime'])


# Conclusion
# Using the network module of the ArcGIS API for Python, you can solve for closest facilities from an incident location.